# Scrape and Clean Data

This program collects season long data for teams in the Power 5 conferences, and the big east for the years 2014 through 2019.  There are 75 total teams, thus, this scrypt scrapes team data from 375 different webpages , availible on Sports-reference.com.   
Additional scraping is done to determine the Win-Loss percentage of each team accross the 6 years of play. This requires a scrape of an additional 75 websites. 
Finaly, the conference tournament games and their results are scaped. 

Once the data is collected, it is organized such that for each of the conference tournament games in the past six years, each tournament game, and its result is written into a  pandas dataframe, with the season stats of each team provided as well.

All collected data is from Sports-reference.com

##  Scrape the Data

In [1]:
# imports we want
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import lxml.html as lh
import numpy as np
import matplotlib.pyplot as plt
import csv

####### useful 
#df.loc[row_selection, col_selection]
########

In [2]:
## this will get the games played in tournament play
## this will get the standings for each team after postseason play, and provide a list of all big east teams in the conference over time
powerSixConf = ['big-east' , 'big-ten' , 'acc' , 'sec' , 'pac-12' , 'big-12'] # power six conference
powerSixConfID = [1 ]      # ID's of the power Six Conf
for k in range(1 , len(powerSixConf)):
    powerSixConfID += [powerSixConfID[k-1] + 1]
powerSixTeams = []    # empty list which will be iterated into   
powerSixTeamsAndConf = []   # we now also add in the conference ID
conferenceTournamentGames = pd.DataFrame()    # empty dataframe


for i in range(2015 , 2020 , 1):
    for j in range(len(powerSixConf)):
    
        url = "https://www.sports-reference.com/cbb/conferences/" + str(powerSixConf[j]) + "/" + str(i) + "-schedule.html"   # tournament games
        #url2 = "https://en.wikipedia.org/wiki/" + str(i) + "_SEC_Men%27s_Basketball_Tournament"    # league standings
        url2 = "https://www.sports-reference.com/cbb/conferences/" + str(powerSixConf[j]) + "/" + str(i) + ".html"       # league standings
    #read the data with Beautiful SOup
    ############ this is for the tournament games  ##############  
    #this can be iterated over a number of conferences as well
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "lxml")
        tables = soup.findAll('table')

    # make the dataframe for the tournament data 
        df = pd.read_html(str(tables))[0]

    # we need to clean the data for the tournament 
        when = df[df['Notes']=='Notes'].index.values.astype(int)
        for k in range(len(when)):	
            df.drop(when[k] , inplace=True)
        notes = df['Notes']
    # this will only get the tournament games
        when2 = notes.first_valid_index()
        df2 = df.loc[when2:]

    ### now lets add on a column of the year
        numRows = len(df2.index)
        currYear = [i] * numRows
        df2['Date'] = currYear   # we only care about the year not the date
        conferenceTournamentGames = conferenceTournamentGames.append(df2 , ignore_index=True) 


    ## this is for the standings 
        pageSt = requests.get(url2)
        soupSt = BeautifulSoup(pageSt.content, "lxml")
        tablesSt = soupSt.find('table')


    # make the dataframe for the standings data
        df2 = pd.read_html(str(tablesSt))[0]

        standings = df2[df2.columns[[0,1]]]

        ## this will generate a list of all the teams that are in the power six conf for the years in question
        for p in range(len(standings.index)):
            team = standings.iloc[p,1]
            team = team.lower()
            team = team.replace(' ' , '-')
            team = team.replace('(' , '')
            team = team.replace(')' , '')
            team = team.replace('.' , '')
            team = team.replace( "'" , '')
            team = team.replace("&" , "")
            team = team.replace("university-of-" , "")            
            if team not in powerSixTeams:
                powerSixTeams += [team]
            combo = [team , powerSixConfID[j]]
            if (combo not in powerSixTeamsAndConf) and (i == 2018):   #this accounts for teams switching conferences
                powerSixTeamsAndConf += [[team , powerSixConfID[j]]]


    # we should clean the team names in the conferenceTournamentGames df in the same way
        for m in range(len(conferenceTournamentGames.index)):
            team1 = conferenceTournamentGames.iloc[m,1]
            team1 = team1.lower()
            team1 = team1.replace(' ' , '-')
            team1 = team1.replace('(' , '')
            team1 = team1.replace(')' , '')
            team1 = team1.replace('.' , '')
            team1 = team1.replace( "'" , '')
            team1 = team1.replace("&" , "") 
            team1 = team1.replace("university-of-" , "")  
            conferenceTournamentGames.iloc[m,1] = team1

            team2 = conferenceTournamentGames.iloc[m,3]
            team2 = team2.lower()
            team2 = team2.replace(' ' , '-')
            team2 = team2.replace('(' , '')
            team2 = team2.replace(')' , '')
            team2 = team2.replace('.' , '')
            team2 = team2.replace( "'" , '')
            team2 = team2.replace("&" , "")   
            team2 = team2.replace("university-of-" , "")  
            conferenceTournamentGames.iloc[m,3] = team2

    
    
    
    

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
# a loop to get the win loss for the teams
WLTeams = pd.DataFrame()      # initialize and empty dataframe
for j in range(len(powerSixTeams)): 
    url3 = "https://www.sports-reference.com/cbb/schools/" + str(powerSixTeams[j]) + "/"
    teamNames = powerSixTeams[j] 
        # get the data
    pageWL = requests.get(url3)
    soupWL = BeautifulSoup(pageWL.content, "lxml")
    tablesWL = soupWL.findAll('table')
        
        # build a dataframe of the win loss data
    win_loss_df = pd.read_html(str(tablesWL))[0]
    win_loss_6_df = win_loss_df.head(5)
    win_loss_6_df['team'] = teamNames

    WLTeams = WLTeams.append(win_loss_6_df , ignore_index=True)    
# this will get the W-L%
    WL = win_loss_df.iloc[:, [5]]

    WL20 = WL.values[1:6]
        # this gets rid of the wierd index This should be done in the dataframe ahead of time 
    index = np.argwhere(WL20=='W-L%')
    WL20 = np.delete(WL20, index)
    
# Add a column of the years to the win loss dataframe
years = [2019 , 2018 , 2017 , 2016 , 2015 ] * 75     
WLTeams['year'] = years
# going to need to use an intersection later to add the win loss record for each team for each year to the dataframe

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
# all we are curious about here are the win loss for out of conference and in conference
WLTeams.drop(WLTeams.columns[[0, 1 , 2, 3 , 4, 6 , 7 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 ]], axis=1 , inplace =True)
WLTeams.columns = ['WL' , 'WLC' , 'teams' , 'year']

In [ ]:
# Provide Team IDS's to each team
powerSixTeamID = [1000]    #  list initiated at 1000 for team ID's            
for i in range(1, len(powerSixTeams)):
    powerSixTeamID += [powerSixTeamID[i-1] + 1]    # this will give ID's to the teams of interest

In [ ]:
# this will collect the season data for every team we selected in the list, for the number of indicated years
# these for loops will scrape data from 450 different webpages
seasonStats = pd.DataFrame()   # start with an empty dataframe
for i in range(2015 , 2020, 1):   # this is the years 
	for j in range(len(powerSixTeams)): # teams
		url3 = "https://www.sports-reference.com/cbb/schools/" + str(powerSixTeams[j]) + "/" + str(i) + ".html" # season data
		pageSe = requests.get(url3)   # scrape
		soupSe = BeautifulSoup(pageSe.content, "lxml")   # parse
		tablesSe = soupSe.findAll('table')    # search for tables 
	    # convert to a dataframe and label the data
		print(i , j)   # this shows progress
		dfSe = pd.read_html(str(tablesSe[1]))[0]      # select the table of interest into a pandas dataframe
		dfSe.drop([1 , 2, 3] , inplace=True)	      # only want the team data. (although might consider fouls against too)
		dfSe = dfSe.replace('Team' , powerSixTeams[j])   # want the name of the team 
		numRows = len(dfSe.index) # get the number of rows...... should just be one here
		dfSe['Team_ID'] = powerSixTeamID[j]        # put in the team ID's
		dfSe['Conf_ID'] = powerSixTeamsAndConf[j][1]        # put in the conference ID's  
		dfSe['Conf'] = powerSixConf[powerSixTeamsAndConf[j][1] - 1]       # put in the team's conference
		currYear = [i] * numRows  ## get the correct number 
		dfSe['Date'] = currYear   # only want the year in this column
		seasonStats = seasonStats.append(dfSe , ignore_index=True) # add this to the season stats empty dataframe we started with 
seasonStats = seasonStats.rename(columns={'Unnamed: 0': 'Team'})  # simply clean up the columns 



2015 0
2015 1
2015 2
2015 3
2015 4
2015 5
2015 6
2015 7
2015 8
2015 9
2015 10
2015 11
2015 12
2015 13
2015 14
2015 15
2015 16
2015 17
2015 18
2015 19
2015 20
2015 21
2015 22
2015 23
2015 24
2015 25
2015 26
2015 27
2015 28
2015 29
2015 30
2015 31
2015 32
2015 33
2015 34
2015 35
2015 36
2015 37
2015 38
2015 39
2015 40
2015 41
2015 42
2015 43
2015 44
2015 45
2015 46
2015 47
2015 48
2015 49
2015 50
2015 51
2015 52
2015 53
2015 54
2015 55
2015 56
2015 57
2015 58
2015 59
2015 60
2015 61
2015 62
2015 63
2015 64
2015 65
2015 66
2015 67
2015 68
2015 69
2015 70
2015 71
2015 72
2015 73
2015 74
2016 0
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
2016 13
2016 14
2016 15
2016 16
2016 17
2016 18
2016 19
2016 20
2016 21
2016 22
2016 23
2016 24
2016 25
2016 26
2016 27
2016 28
2016 29
2016 30
2016 31
2016 32
2016 33
2016 34
2016 35
2016 36
2016 37
2016 38
2016 39
2016 40
2016 41
2016 42
2016 43
2016 44
2016 45
2016 46
2016 47
2016 48
2016 49
2016 50
2016 51
2016

In [ ]:
# lets reorder the columns like we want to 
# moves the date to the first columm 
cols = seasonStats.columns.tolist()
newCols = []
newCols += [cols[-1]]  + [cols[0]] +  [cols[-4]] + [cols[-2]] +[cols[-3]] + cols[1:-4]

In [ ]:
seasonStats = seasonStats[newCols]

## The following few commands Provide visual representation of the data collected 

In [ ]:
WLTeams

In [ ]:
seasonStats

In [ ]:
conferenceTournamentGames

## Re-Organize Our Data into a useful dataframe

In [ ]:
## let's add the Win Loss percentages to the Season Stats DF
# now we need to combine in a helpful way. Let's put the season stats next to each team in the games dataframe.
team1DataDF = pd.DataFrame()    # placeHolder
for i in range(len(seasonStats.index)):
    year  = seasonStats.iloc[i,0]      # get year
    team = seasonStats.iloc[i,1]       # get team
    
    yearIndex = WLTeams[WLTeams['year']==year].index.values.astype(int)
    teamIndex = WLTeams[WLTeams['teams']==team].index.values.astype(int)  


    ## intersection
    
    team1Index= set(yearIndex).intersection(teamIndex).pop()
    placeHolder1 = pd.DataFrame([WLTeams.iloc[team1Index]])   # convert the desired data row into a dataframe
    team1DataDF = team1DataDF.append(placeHolder1 ,  ignore_index=True)   
seasonStats = pd.concat([seasonStats, team1DataDF], axis=1)

In [ ]:
seasonStats.drop(['teams' , 'year'] , axis = 1 , inplace = True)

In [ ]:
# now we need to combine in a helpful way. Let's put the season stats next to each team in the games dataframe.
final = pd.DataFrame()    # this is our final dataframe which we will output 
team1DataDF = pd.DataFrame()    # placeHolder
team2DataDF = pd.DataFrame()    # placeHolder
for i in range(len(conferenceTournamentGames.index)):
    year  = conferenceTournamentGames.iloc[i,0]
    team1 = conferenceTournamentGames.iloc[i,1]
    team2 = conferenceTournamentGames.iloc[i,3]
    yearIndex = seasonStats[seasonStats['Date']==year].index.values.astype(int)
    firstIndex = seasonStats[seasonStats['Team']==team1].index.values.astype(int)  
    secondIndex = seasonStats[seasonStats['Team']==team2].index.values.astype(int)  

    ## intersect of first two for team 1
    ## intersect of first and third for team2
    
    team1Index= set(yearIndex).intersection(firstIndex).pop()
    team2Index= set(yearIndex).intersection(secondIndex).pop()
 
    placeHolder1 = pd.DataFrame([seasonStats.iloc[team1Index]])   # convert the desired data row into a dataframe
    placeHolder2 = pd.DataFrame([seasonStats.iloc[team2Index]])  # same for team two
    team1DataDF = team1DataDF.append(placeHolder1 ,  ignore_index=True) 
    team2DataDF = team2DataDF.append(placeHolder2 ,  ignore_index=True)   

# Add .1 to all the second team data, except the index labeled 'Date'
names = ['Date']   # need to start with date in it. 
colNames = team2DataDF.columns
for j in range(1 , len(colNames) , 1):
    add = ".1"
    val = colNames[j] + add
    names = names + [val]
team2DataDF.columns = names   # this puts it back in as column indexes
    
# get the score of each tournament game
third = pd.DataFrame(conferenceTournamentGames[conferenceTournamentGames.columns[[2,4]]])
third.columns = ['Score' , 'Score.1']  
# concatenate them together in the correct order
final = pd.concat( ( team1DataDF  ,  team2DataDF , third ) ,  axis=1 , sort = False)   # we have one extra date column 
final = final.loc[:,~final.columns.duplicated()]  # this removes the duplicates. This is the reason we kept 'Date' as is earlier




## Visual representation of the Final DataFrame


The dataframe shown below will be imported into the next python scrypt. Machhine learning will be used to generate odds for a any game based off of the teams season long statistics.

In [ ]:
final

## Write the dataframes to a CSV file

In [ ]:
# this sends our data to a csv file
final.to_csv('PowerSixBball.csv', sep='\t', encoding='utf-8')
seasonStats.to_csv('SeasonStatistics.csv' , sep='\t', encoding='utf-8')
conferenceTournamentGames.to_csv('conference_tournament_games.csv' , sep='\t', encoding='utf-8')

